In [1]:
import tensorflow as tf
from GAN import GAN
import numpy as np
import os
import matplotlib.pyplot as plt
import mnist
from skimage.transform import resize, rotate, warp, SimilarityTransform
%matplotlib inline

In [2]:
mn = mnist.Mnist(path=r"C:\Users\srajabza\Documents\mnist")
imgs, lbls, _ = mn.get_batch(60000)
imgs = imgs[list(np.where((lbls==0)|(lbls==1)|(lbls==2)))[0],...]
imgs = (imgs - 0.5)*2
assert(np.all((imgs>=-1)&(imgs<=1)) and np.any(imgs>0) and np.any(imgs<0))

Images magic no: 2051,  No of images: 60000, Image rows: 28, Image cols: 28
Labels magic no: 2049, No of items: 60000


In [3]:
def transform_image(img, code):
    if code == 0:
        return(img)
    
    if code == 1:
        return(rotate(img, np.random.randint(low=-10, high=10)))
    
    if code == 2:
        return(img + np.random.normal(size=img.shape, loc=0, scale=0.1))
    
    
    if code == 3:
        sft = np.random.randint(-4, 4)
        tr = SimilarityTransform(scale=1, rotation=0, translation=[sft,sft])
        img = warp(img, tr)
        if sft>0:
            img[:,-sft::] = -1
            img[-sft::,:] = -1
        else:
            img[:,0:-sft] = -1
            img[0:-sft,:] = -1
        return(img)
        

In [10]:
def get_batch(batch_size):
    global imgs
    rnd_idxs = np.random.choice(np.arange(imgs.shape[0]), batch_size)
    for idx in rnd_idxs:
        code = np.random.choice(np.arange(4), 1)[0]
        imgs[idx] = transform_image(imgs[idx], code=code)
    return (imgs[rnd_idxs])    
    
        

In [8]:
z_dim = 100

tf.reset_default_graph()
gan = GAN(verbose=True)
layers = list()

ns = tf.train.exponential_decay(0.1, tf.Variable(0, trainable=False), 1000, 0.95, staircase=True)

G_layers = [[1024, [4, 4], (1, 1), "valid", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [512, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [256, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [1, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 0, "th", None, 0]]

D_layers = [[256, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 0, 'lr', None, ns],
            [512, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 1, 'lr', None, 0],
            [1024, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 1, 'lr', [4, 5], 0],
            [1, [4, 4], (1, 1), 'valid', tf.random_normal_initializer(stddev=0.02), 0, 'l', None, 0]]

x = tf.placeholder(tf.float32, [None, 28, 28])
xr = tf.image.resize_images(tf.expand_dims(x, axis=3), (32, 32))

gargs = gan.build_graph(G_layers, D_layers, xr,
                       tf.placeholder(tf.float32, [None, 1, 1, z_dim]), True, 0.9, ["RMS", 0.5, 0.99], 2e-4, True)
gargs = [g for g in gargs]
gargs[1] = x
gargs = tuple(gargs)

=====generator=====
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
-layer:0, no_of_filters:1024, k_size:[4, 4], strides:(1, 1), padding:valid, initializer:<tensorflow.python.ops.init_ops.RandomNormal object at 0x0000020F5B0B6780>, has_bn:1, act:r output shape:[None, 4, 4, 1024] mb:None noise:0-
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
-layer:1, no_of_filters:512, k_size:[4, 4], strides:(2, 2), padding:same, initializer:<tensorflow.python.ops.init_ops.RandomNormal object at 0x0000020F5AF82240>, has_bn:1, act:r output shape:[None, 8, 8, 512] mb:None noise:0-
----------------------------------------------------------------------

In [ ]:
gan.train(60,  2000, 10, get_batch, "n", z_dim, True, "summary\\GAN_mnist", "Model\\GAN_mnist", None, None, gargs)


Training Started!
 epoch 10 cnt 10 Loss discriminator 0.5882760286331177   Loss generator 3.3088605403900146 